<a href="https://colab.research.google.com/github/jay24rajput/Covid19-chest-xray-prediction/blob/master/Chest_XRay_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the data 

In [1]:
! git clone https://github.com/ieee8023/covid-chestxray-dataset.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3687, done.
remote: Total 3687 (delta 0), reused 0 (delta 0), pack-reused 3687
Receiving objects: 100% (3687/3687), 715.78 MiB | 23.42 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
Checking out files: 100% (1152/1152), done.


## Getting all the required images

In [2]:
import os 
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('covid-chestxray-dataset/metadata.csv')

In [4]:
df

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,in_icu,needed_supplemental_O2,extubated,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,view,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
1,2,3.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
2,2,5.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
3,2,6.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
4,4,0.0,F,52.0,COVID-19,Y,NaN,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,468,NaN,F,50.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,X-ray,NaN,"Adelaide, Australia",images,e0e3a6526a3fecadfca2be13242798_jumbo.jpg,NaN,https://radiopaedia.org/cases/left-lower-lobe-...,CC BY-NC-SA,"Shortness of breath, fever and rigors Opacity ...","Case courtesy of Dr Thuan Tzen, Koh, Radiopaed..."
926,469,NaN,F,25.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,NaN,"Calgary, Canada",images,2264f643b18b1010ec10a850f17550_jumbo.jpeg,NaN,https://radiopaedia.org/cases/silhouette-sign-...,CC BY-NC-SA,A young woman is sent with a diagnosis of pneu...,"Case courtesy of Dr Garth Kruger, Radiopaedia...."
927,469,NaN,F,25.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,X-ray,NaN,"Calgary, Canada",images,800f798a58d0cbcc72eb234f192461_jumbo.jpeg,NaN,https://radiopaedia.org/cases/silhouette-sign-...,CC BY-NC-SA,A young woman is sent with a diagnosis of pneu...,"Case courtesy of Dr Garth Kruger, Radiopaedia...."
928,470,NaN,M,55.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,NaN,"Melbourne, Australia",images,55f5189d2c23688ac8dc1d58eb65cf_jumbo.jpg,NaN,https://radiopaedia.org/cases/left-lower-lobe-...,CC BY-NC-SA,Productive cough and fever Region of consolida...,"Case courtesy of Dr Andrew Dixon, Radiopaedia...."


## Make Folder for covid positive and negative

In [5]:
os.makedirs('training/positive')
os.makedirs('training/negative')
os.makedirs('validation/positive')
os.makedirs('validation/negative')

## Analyze the dataframe and transfer images to folder

In [6]:
from shutil import copy
repository_path = 'covid-chestxray-dataset'
positive_images = []
negative_images = []
for index in df.index:
  image_path = os.path.join(repository_path,df['folder'][index],df['filename'][index])
  if image_path.endswith('.jpg') or image_path.endswith('.jpeg') or image_path.endswith('.png'):
    if df['finding'][index] == 'COVID-19':
      positive_images.append(image_path)
    else:
      negative_images.append(image_path)

In [7]:
print('Postive samples:',len(positive_images))
print('Negative samples:',len(negative_images))

Postive samples: 536
Negative samples: 367


## Classify Images into train and test Director

In [8]:
import random
training_percentage = 0.9

# shuffle images randomly 
random.shuffle(positive_images)
random.shuffle(negative_images)

In [9]:
positive_training_length = int(training_percentage * len(positive_images))
negative_training_length = int(training_percentage * len(negative_images))

print(positive_training_length, negative_training_length)

482 330


In [10]:
def make_directories(type):
  if type == 'training':
    positive = positive_images[:positive_training_length]
    negative = negative_images[:negative_training_length]
  elif type == 'validation':
    positive = positive_images[positive_training_length:]
    negative = negative_images[negative_training_length:]

  # copy positive images
  destination = os.path.join(type, 'positive')
  for image_path in positive:
    copy(image_path,destination)

  # copy negative images
  destination = os.path.join(type, 'negative')
  for image_path in negative:
    copy(image_path, destination)

In [11]:
make_directories('training')
make_directories('validation')

## Make Image Data Generator

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'training'
validation_dir = 'validation'

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 812 images belonging to 2 classes.
Found 91 images belonging to 2 classes.


## Define the Model

In [22]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 

    # tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    # tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(), 
    
    tf.keras.layers.Dense(256, activation='relu'), 

    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

## Compile the Model

In [23]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [24]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 41,
            epochs = 50,
            validation_steps = 5,
            verbose = 1)

Epoch 1/50
41/41 [==============================] - 35s 860ms/step - loss: 0.8091 - accuracy: 0.5530 - val_loss: 0.6453 - val_accuracy: 0.5934
Epoch 2/50
41/41 [==============================] - 35s 848ms/step - loss: 0.6943 - accuracy: 0.5924 - val_loss: 0.6472 - val_accuracy: 0.5934
Epoch 3/50
41/41 [==============================] - 35s 844ms/step - loss: 0.7002 - accuracy: 0.5837 - val_loss: 0.6789 - val_accuracy: 0.5604
Epoch 4/50
41/41 [==============================] - 35s 847ms/step - loss: 0.7030 - accuracy: 0.5899 - val_loss: 0.6902 - val_accuracy: 0.4945
Epoch 5/50
41/41 [==============================] - 35s 851ms/step - loss: 0.6707 - accuracy: 0.5837 - val_loss: 0.6259 - val_accuracy: 0.5934
Epoch 6/50
41/41 [==============================] - 35s 856ms/step - loss: 0.6804 - accuracy: 0.5998 - val_loss: 0.6414 - val_accuracy: 0.6044
Epoch 7/50
41/41 [==============================] - 35s 850ms/step - loss: 0.6726 - accuracy: 0.6010 - val_loss: 0.6419 - val_accuracy: 0.5934

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [26]:
model.save('/content/drive/My Drive/Models/covid-chest-xray.h5')